In [ ]:
#%load_ext autoreload
#%autoreload 2
#%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import inspect

print(np.__version__)

In [ ]:
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [ ]:
# Setting data location
eeg_data_path = '/Users/siyitang/Documents/RubinLab/Project/EEG/Data'
eeg_data_file = 'reports_unique_for_hl_mm.xlsx'
data_path = os.path.join(eeg_data_path, eeg_data_file)

# Loading data
df_eeg = pd.read_excel(data_path, index_col=0).dropna(how='all')

# Rename the column names
df_eeg = df_eeg.rename(index=str, columns={"Hand Label (1 for seizure, -1 for no seizure, 0 for unsure)": "hand_label","Note":"note"})
print(df_eeg.columns.values)

In [ ]:
# Number of rows
print('Number of rows:{}'.format(len(df_eeg)))

In [ ]:
# Print a single note for understanding
idx = np.random.randint(10342)
idx = 2204
# row index in spreadsheet - 2
print(idx)
print(df_eeg['mrn'][idx])
print(df_eeg['note'][idx])

In [ ]:
# Convert to EEGNote
from eeg_utils import EEGNote
noteObj = EEGNote(df_eeg['note_uuid'][idx], df_eeg['note'][idx])

In [ ]:
# Split into sections dictionary
note_sec = noteObj.create_sections_dict()
print('Number of sections: {}'.format(len(note_sec)))
#print('Index: {}'.format(idx))
print(note_sec)

In [ ]:
for key, val in note_sec.items():
    print('{}: {}\n'.format(key, val))

# Test extracting exact time of seizures

In [ ]:
# If seizures/events section exists
SEIZURE_EVENTS_SYN = ['seizures/events', 'seizures:', 'findings', 'push button events', 'neurobehavioral events']
seizures_events_txt = ''
for name in SEIZURE_EVENTS_SYN:
    if name in note_sec.keys() and len(note_sec[name]) != 0:
        seizures_events_txt = note_sec[name]
        break

print(seizures_events_txt)

In [ ]:
import re
from eeg_utils import extract_major_note_sections

#TODO: Add more patterns for time format
## NOTE: NEED TO PUT MORE SPECIFIC FORMATS IN FRONT OF MORE GENERAL FORMATS!!
#DATE_TIME_SEC = r'(\d+)/(\d+)\,\s(\d+):(\d+):|(\d+)/(\d+)/(\d+)\s(\d+):(\d+):(\d+)'
TIME2TIME_SEC = r'(\d+):(\d+):(\d+)\s-\s(\d+):(\d+):(\d+)|(\d+)\s-\s(\d+)'
#NUMBERING_SEC = r'\d\)|\(\d\)'
TIME_ONLY = r'(\d\d\d\d)\s|(\d\d):(\d\d):(\d\d)\s[A-Z]|(\d\d):(\d\d):(\d\d):|(\d\d):(\d\d):[!0-9][!0-9]|(\d\d):(\d\d)\s[A-Z]'
DATE_ONLY = r'(\d+)/(\d+)/(\d+)'
EXAMPLE_TIME = r'(\d\d\d\d)|(\d+):(\d+):(\d+)\s-\s(\d+):(\d+):(\d+)|(\d+):(\d+)-(\d+):(\d+)|(\d+):(\d+):(\d+)|(\d+):(\d+)|(\d+):(\d+):(\d+)\s(and)\s(\d+):(\d+):(\d+)|(\d+):(\d+)\s(and)\s(\d+):(\d+)|(\d+)\s(and)\s(\d+)'

## Some seizure detection phrases
# 4 seizures were noted
# examples occur at
# occur 2-3 times
# at least 39 seizures were recorded
# seizure detection
SEIZURE_DETECT_PHRASE = r'seizure\sdetection|examples|(\d+)\sseizures|seizure|seizures|seizure\srecorded|seizure\sonset|spasms|myoclonic|clonic'
SEIZURE_DETECT_PHRASE_RE = re.compile(SEIZURE_DETECT_PHRASE, re.IGNORECASE)
NO_SEIZURE_PHRASE = r'no\sseizures|no\sseizure'
NO_SEIZURE_PHRASE_RE = re.compile(NO_SEIZURE_PHRASE, re.IGNORECASE)

# Extract subsections first if exist, then extract example times
if re.search(DATE_ONLY, seizures_events_txt):
    time_format = DATE_ONLY
elif re.search(TIME2TIME_SEC, seizures_events_txt):
    time_format = TIME2TIME_SEC
elif re.search(TIME_ONLY, seizures_events_txt):
    time_format = TIME_ONLY
elif re.search(EXAMPLE_TIME, seizures_events_txt):
    time_format = EXAMPLE_TIME
else:
    time_format = None
print('Time format:{}'.format(time_format))
if time_format is not None:
    times_re = re.finditer(time_format, seizures_events_txt)
    times = [seizures_events_txt[idx.start():idx.end()] for idx in times_re]
    times_idx = [(idx.start(), idx.end()) for idx in times_re]
    print('Times: {}'.format(times))
    
seizure_times = []  
seizure_sections = {}
if time_format != EXAMPLE_TIME and time_format is not None:   
    sections = extract_major_note_sections(seizures_events_txt, times)
    print('All sections: {}'.format(sections))
    # Determine whether each time period has seizures detected
    for time, event in sections.items():
        if time_format == DATE_ONLY:
            if re.search(TIME_ONLY, event) or re.search(TIME2TIME_SEC, event): # break down into more fine-grained times
                if re.search(TIME2TIME_SEC, event):
                    times_subsec_re = re.finditer(TIME2TIME_SEC, event)
                else:
                    times_subsec_re = re.finditer(TIME_ONLY, event)
                times_subsec = [event[idx.start():idx.end()] for idx in times_subsec_re]
                subsections = extract_major_note_sections(event, times_subsec)
                for ky, val in subsections.items():
                    if SEIZURE_DETECT_PHRASE_RE.search(val) and not NO_SEIZURE_PHRASE_RE.search(val):
                        seizure_sections[time + ' ' + ky] = val
                        seizure_times.append(time + ' ' + ky)
            elif re.search(EXAMPLE_TIME, event): # search for example times
                eg_time_re = re.finditer(EXAMPLE_TIME, event)
                eg_times = [event[idx.start():idx.end()] for idx in eg_time_re]
                seizure_sections[time] = event
                seizure_times.extend(eg_times)
        else:
            if SEIZURE_DETECT_PHRASE_RE.search(event) and not NO_SEIZURE_PHRASE_RE.search(event):
                seizure_sections[time] = event                
                seizure_times.append(time)     
elif time_format == EXAMPLE_TIME: # entire section is one paragraph with some examples of seizure onset times
    if SEIZURE_DETECT_PHRASE_RE.search(seizures_events_txt) and not NO_SEIZURE_PHRASE_RE.search(seizures_events_txt):
        seizure_times.extend(times)
               
print('All found times: {}'.format(seizure_times))
print('Seizure sections: {}'.format(seizure_sections))

# Multi-class seizure classification
**Look at "INTERPRETATION", "SUMMARY", "Impression", "Indication" and the above extracted seizure events**
## Motor seizures
**A generic term, can be either generalized or focal onset seizures.**
* **Tonic**: "tonic", e.g. 210, 219, 238, 251, 273, 356
* **Clonic**: "clonic", e.g. 191, 193, 197, 206, 216, 273, 247
* **Tonic-clonic and variants**: "tonic clonic", "tonic-clonic", e.g. 369, 238
* **Atonic**: "atonic", e.g. 216
* **Myoclonic**: "myoclonic", "myoclonia", e.g. 181, 283, 351
* **Infantile spasm**: "infantile spasms", e.g. 192, 255, 257, 369
* **Epileptic spasm**: "epileptic spasms", e.g. 1232, 1937, 2003
* **Focal motor seizure with paresis/paralysis**: "paralyzed", "paralysis", e.g. 3995, 4916
* **Focal motor seizure with dystonia**: "dystonic", "dystonia", e.g. 251
* **Automatism**: "automatisms", e.g. 186, 8523, 8820, 8930
* **Focal motor seizure with dysarthria / anarthria**: "dysarthria" but not "no dysarthria", e.g. 5008
* **Focal hyperkinetic seizure**: "hypermotor", "hyperkinetic", e.g. 323, 8195, 9214, 1726, 3979
* **Focal motor seizure with negative myoclonus**: "negative myoclonus", e.g. 5193, 8438, 9197
* **Focal motor seizure with version**: "version", e.g. 5161, 5280, 5319
* **Focal bilateral motor seizure (This type of seizure is also described as a 'fencer's posture' or a 'figure of 4')**: "fencing posture", "bilateral motor", "figure of 4", "fencer", e.g. 197, 1909, 5024, 939, 1882
* **Epilepsia partialis continua**: "epilepsia partialis continua", e.g. 2486, 4919, 8467


## Hyperkinetic seizures
**This seizure type involves movements of proximal limb or axial muscles, producing irregular large amplitude movements, such as pedaling, pelvic thrusting, jumping, thrashing and/or rocking movements.**
https://www.epilepsydiagnosis.org/seizure/motor-overview.html#hyperkinetic
* "hypermotor", "hyperkinetic"
* e.g. 323, 9214, 1726

## Clonic seizures
* "clonic", but not "no clonic" or "tonic clonic" or "tonic-clonic"
* e.g. 191, 193, 197, 206, 216, 273, 247

## Tonic seizures
* "tonic", but not "no tonic" or "tonic clonic" or "tonic-clonic"
* e.g. 210, 219, 238, 251, 273, 356


## Based on behaviors
* **Stiffening**: "stiff", "stiffening", e.g. 189, 191
* **Clonic jerking**: "clonic jerk", e.g. 273, 283, 247
* **Loss of tone**: NO EXAMPLES FOUND
* **Dystonic posturing**: "dystonic", e.g. 251
* **Figure of Four**: NO EXAMPLES FOUND
* **Fencing posture**: "fencing posture", e.g. 197
* **Automatisms**: "automatisms", e.g. 186

## Based on impression of seizure types
* **Tonic-clonic and variants**: "tonic clonic", "tonic-clonic", e.g. 369, 238
* **Tonic**: "tonic", e.g. 210, 219, 238, 251, 273, 356
* **Atonic**: "atonic", e.g. 216
* **Myoclonic**: "myoclonic", "myoclonia", e.g. 283, 351
* **Infantile spasm**: "infantile spasms", e.g. 192, 255, 257, 369
* **Epileptic spasm**: "epileptic spasms", NO EXAMPLES FOUND
* **Absence seizure**: "absence seizures", "absence-like", "absence epilepsy", e.g. 179, 186, 187, 198, 205, 208, 212, 213, 300, 333, 334
* **Focal seizure impairing awareness (or complex partial seizure)**: "complex partial seizures", e.g. 196, 218, 221, 230
* **Frontal seizure**: "frontal", "frontal seizures", e.g. 283, 194
* **Temporal seizure**: "temporal", "temporal seizures", e.g. 191
* **Parietal seizure**: "parietal", "parietal seizures", NO EXAMPLES FOUND
* **Occipital seizure**: "occipital", "occipital seizures", e.g. 193, 197
* **Gelastic seizure (a type of emotional seizures)**: "gelastic seizures", e.g. 201
* **Dacrystic seizure (a type of emotional seizures)**: "dacrystic seizures", NO EXAMPLES FOUND

